In [1]:
import Pkg
Pkg.activate(".")

  Activating project at `~/Developer/DistributedStreams`


In [2]:
using Distributed

In [3]:
addprocs(2)

2-element Vector{Int64}:
 2
 3

In [4]:
@everywhere using Base: @kwdef
@everywhere using Distributed, DistributedStreams, DistributedArrays

Precompiling DistributedStreams
  ✓ DistributedStreams
  1 dependency successfully precompiled in 2 seconds. 19 already precompiled.


In [5]:
control_messages, control_responses, distributed_control, status = launch_sentinel(;
    workers=[2], verbose=true, buffer_size=1024, timeout=1
)

(RemoteChannel{Channel{ControlMessage}}(1, 1, 19), RemoteChannel{Channel{ControlMessage}}(1, 1, 20), @NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}[(worker = 2, flag = Base.RefValue{Bool}(false))], Any[Future(2, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 139994581224544, 139994735829000)), nothing)])

In [6]:
distributed_control

1-element DArray{@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}}}:
 (worker = 2, flag = Base.RefValue{Bool}(false))

In [8]:
status

1-element Vector{Any}:
 Future(2, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 139994581224544, 139994735829000)), nothing)

In [9]:
ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

@everywhere function test_fn(i)
    println("hi there, I'm running on pid=$(myid())")
    sleep(2)
    return i+1
end

In [10]:
m_f = ControlMessage(
    message_type=DistributedStreams.start,
    target=3,
    func_f=test_fn,
    func_in=ch_in,
    func_out=ch_out
)

ControlMessage(DistributedStreams.start, 3, test_fn, RemoteChannel{Channel{Int64}}(1, 1, 28), RemoteChannel{Channel{Int64}}(1, 1, 29))

In [11]:
put!(control_messages, m_f)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

      From worker 2:	Start instruction for 3
      From worker 2:	Started


In [12]:
collect!(control_responses)

ControlMessage[]

In [13]:
put!(ch_in, 10)

RemoteChannel{Channel{Int64}}(1, 1, 28)

      From worker 3:	hi there, I'm running on pid=3


In [14]:
collect!(ch_out)

1-element Vector{Int64}:
 11

In [15]:
status

1-element Vector{Any}:
 Future(2, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 139994581224544, 139994735829000)), nothing)

In [18]:
fetch(status[1])

LoadError: On worker 2:
MethodError: [0mCannot `convert` an object of type 
[0m  [92mTuple{DArray{@NamedTuple{worker::Int64, safe::Base.RefValue{Bool}, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{worker::Int64, safe::Base.RefValue{Bool}, flag::Base.RefValue{Bool}}}}, Vector{Any}}[39m[0m to an object of type 
[0m  [91mDArray[39m

[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::T[39m) where T
[0m[90m   @[39m [90mBase[39m [90m[4mBase.jl:84[24m[39m
[0m  convert(::Type{T}, [91m::LinearAlgebra.Factorization[39m) where T<:AbstractArray
[0m[90m   @[39m [33mLinearAlgebra[39m [90m~/local/juliaup/juliaup/julia-1.10.1+0.x64.linux.gnu/share/julia/stdlib/v1.10/LinearAlgebra/src/[39m[90m[4mfactorization.jl:108[24m[39m
[0m  convert(::Type{T}, [91m::T[39m) where T<:AbstractArray
[0m[90m   @[39m [90mBase[39m [90m[4mabstractarray.jl:16[24m[39m
[0m  ...

Stacktrace:
 [1] [0m[1msetindex![22m
[90m   @[39m [90m./[39m[90m[4mdict.jl:376[2

In [19]:
status

1-element Vector{Any}:
 Future(2, 1, 21, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (1, 139994581224544, 139994735829000)), nothing)

In [37]:
rmprocs(3)

Task (done) @0x00007f89824ed460

In [21]:
collect!(control_responses)

ControlMessage[]

In [22]:
distributed_control

1-element DArray{@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}, 1, Vector{@NamedTuple{worker::Int64, flag::Base.RefValue{Bool}}}}:
 (worker = 2, flag = Base.RefValue{Bool}(false))

In [17]:
addprocs(1)
@everywhere using Distributed, DistributedStreams, DistributedArrays

In [18]:
# ch_in = RemoteChannel(()->Channel{Int64}(32), 1)
# ch_out = RemoteChannel(()->Channel{Int64}(32), 1)

m = ControlMessage(
    message_type=DistributedStreams.start,
    target=4,
    func_f=m_f.func_f,
    func_in=m_f.func_in,
    func_out=m_f.func_out
)

ControlMessage(DistributedStreams.start, 4, test_fn, RemoteChannel{Channel{Int64}}(1, 1, 28), RemoteChannel{Channel{Int64}}(1, 1, 29))

In [19]:
put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

In [20]:
collect!(control_responses)

ControlMessage[]

In [21]:
m = ControlMessage(
    message_type=DistributedStreams.stop,
    target=4,
    func_f=nothing,
    func_in=nothing,
    func_out=nothing
)

put!(control_messages, m)

RemoteChannel{Channel{ControlMessage}}(1, 1, 19)

In [22]:
collect!(control_responses)

ControlMessage[]

In [23]:
rmprocs(4)

Task (done) @0x00007f653cf3f530

In [24]:
collect!(control_responses)

ControlMessage[]

In [28]:
workers()

1-element Vector{Int64}:
 2

In [29]:
stop_workers!(distributed_control; workers=[2])

      From worker 2:	Sentinel on 2 is shutting down


In [30]:
rmprocs()

Task (done) @0x00007f3c2b37a0e0